In [2]:
from pathlib import Path
import sys
import os

sys.path.append(str(Path.cwd().parent/'maet_pln'))

# set this if running on MacOS
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

from data_handler.data_handler import GPT2DatasetHandler
from tokenization import GPT2TokenizationHandler
from model_builder import GPT2ModelBuilder

/Users/toby/anaconda3/envs/maet-pln/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/toby/anaconda3/envs/maet-pln/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [5]:
tk = GPT2TokenizationHandler()
tk.create_tokenizer()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2TokenizerFast(name_or_path='gpt2-medium', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|pad|>'}, clean_up_tokenization_spaces=True)

In [7]:
dataset_name  = "EdinburghNLP/xsum"
dh = GPT2DatasetHandler(dataset_name, tk.tokenizer, data_size=10000, push_to_hub=True, save_locally=False)
dh.data_to_json()
dh.process_data()

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/toby/.cache/huggingface/token
Login successful


Found cached dataset xsum (/Users/toby/.cache/huggingface/datasets/EdinburghNLP___xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/toby/.cache/huggingface/datasets/json/default-41781da5ec32dbfc/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


...

Token indices sequence length is longer than the specified maximum sequence length for this model (1239 > 1024). Running this sequence through the model will result in indexing errors


Filter:   0%|          | 0/9000 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
cwd = Path.cwd()
cwd

PosixPath('/Users/toby/Dev/maet-pln/notebooks')